This notebook describes the generation of the final demand matrix for Quebec households. It needs data on Quebec households consumption and it needs a technology matrix separating Quebec from Canada (produced in the technolgoy_matrix.ipynb notebook).

Import the libraries

In [1]:
import pandas as pd
import numpy as np
import pymrio

Create the pymrio object:

In [4]:
path_to_exiobase_unzipped_folder = 'my_own_path/exiobase3.4/IOT_2011_pxp/'
io = pymrio.parse_exiobase3(path_to_exiobase_unzipped_folder)

Load the technology matrix created in the technology_matrix.ipynb notebook

In [5]:
A = np.load('one_matrix_to_rule_them_all.npy')

In [6]:
# the storage in npy does not include indexes
index = pd.MultiIndex.from_product([io.get_regions().tolist()+['QC'],io.get_sectors()],names=['region','sector'])
index = index.tolist()+[('QC','grid_mix')]
A = pd.DataFrame(A, index=index, columns=index)

We load the Per_quintile_desagregated tab of the Concordance.xlsx excel file in the Data folder which matches data used from Statistics Quebec to EXIOBASE.

In [10]:
df = pd.read_excel('path_to_mordor/Data/Concordance.xlsx','Per_quintile_desaggregated')

When data was unavailable or negligeable, Statistics Quebec assigned the value 'F'. We set that value to zero.

In [11]:
list_quintile = ['1st_quintile','2nd_quintile','3rd_quintile','4th_quintile','5th_quintile']

commodities = io.get_sectors().tolist()
commodities.append('grid_mix')

# remove the Fs of the quebec stats excel
df = df.mask(df=='F')
df.loc[:,list_quintile] = df.loc[:,list_quintile].fillna(0)

We loop through all quintiles.

For each each quintile, their national (or rather provincial here) consumption in terms of EXIOBASE sector is stored in the dictionary consumption_quintile.

There are some conflicts with classifications which were dealt as follows:
- meat consumption in Quebec is assumed to be the same as the average in Canada
- dairy and eggs were separated using the ratio 0.93:0.07 (https://www.mapaq.gouv.qc.ca/fr/Publications/Bottin_consommation_distribution.pdf)
- heat, water and electricity were supplied by 70% electricity, 7% gas, 8% fuel oil and 15% wood (http://energie.hec.ca/wp-content/uploads/2014/12/EEQ2015_FINAL_2015.pdf). Since water is free in Quebec (for the households) and that we are using IO to model the impacts, it corresponds to a 0% share for water.
- the purchase and use of one's car was split between "Motor vehicles, trailers and semi-trailers", "Motor Gasoline", "Gas/Diesel Oil" and "Sale, maintenance, repair of motor vehicles, motor vehicles parts, motorcycles, motor cycles parts and accessoiries". The same distribution as Canada was used for Quebec.
- for other cases of agregation, the value spent by the household is divided equally between the corresponding EXIOBASE sectors.

The statistics from Quebec do not include the purchase of waste treatment services, the same consumption of waste treatment services as in Canada was adopted for Quebec households.

Similarly, no data on importation by Quebec households was available and we thus adapted it from Canada' households.

We assume that all products are bought and fabricated in Quebec. There is no interaction between Quebec's consumption and other provinces. It constitutes a reasonable assumption as inter-provincial trade is negligible (1% of total imports): http://www.stat.gouv.qc.ca/statistiques/economie/commerce-exterieur/imp_pays.htm

In [17]:
total_Y = pd.DataFrame(0,index=A.index,columns=list_quintile)

for quintile in list_quintile:
    consumption_quintile = {}
    for commodity in commodities:
        consumption_quintile[commodity] = 0
    for product in df.index:
        if product == 'Viande':
            list_viande = ['Products of meat cattle','Products of meat pigs','Products of meat poultry','Meat products nec']
            meat_vector = io.Y.loc[:,('CA','Final consumption expenditure by households')].groupby('sector').sum().loc[list_viande]/io.Y.loc[:,('CA','Final consumption expenditure by households')].groupby('sector').sum().loc[list_viande].sum()*df.loc[product,quintile]
            for meat in meat_vector.index:
                consumption_quintile[meat] = meat_vector[meat]
        elif product == 'Produits laitiers et oeufs':
            consumption_quintile['Dairy products'] = 0.93 * df.loc[product, quintile]
            consumption_quintile['Food products nec'] = 0.07 * df.loc[product, quintile]
        elif product == 'Eau, combustibles et électricité pour logement principal':
            consumption_quintile['grid_mix'] = 0.70 * df.loc[product, quintile]
            consumption_quintile['Distribution services of gaseous fuels through mains'] = 0.07 * df.loc[product, quintile]
            consumption_quintile['Heavy Fuel Oil'] = 0.08 * df.loc[product, quintile]
            consumption_quintile['Products of forestry, logging and related services (02)'] = 0.15 * df.loc[product, quintile]
        elif product == 'Automobiles, fourgonnettes et camions à usage privé':
            list_automobile_expenses = ['Motor vehicles, trailers and semi-trailers (34)','Motor Gasoline','Gas/Diesel Oil','Sale, maintenance, repair of motor vehicles, motor vehicles parts, motorcycles, motor cycles parts and accessoiries']
            for expense in list_automobile_expenses:
                consumption_quintile[expense] = (io.Y.loc[[i for i in io.Y.index if i[1] == expense],('CA','Final consumption expenditure by households')].sum()/
                                  io.Y.loc[[i for i in io.Y.index if i[1] in list_automobile_expenses],
                                           ('CA','Final consumption expenditure by households')].sum()*
                                  df.loc[[product,"Utilisation d'automobiles, de fourgonnettes et de camions"],quintile].sum())
        # we just aggregated buying cars with using cars, so pass cause it's already included
        elif product == "Utilisation d'automobiles, de fourgonnettes et de camions":
            pass
        else:
            try:
                if np.isnan(df.loc[product, df.columns[-3]]):
                    consumption_quintile[df.loc[product, 'EXIOBASE sector(s)']] += df.loc[product, quintile]
            except:
                # if it pops an error it means we have to split the purchases
                try:
                    if np.isnan(df.loc[product, df.columns[-2]]):
                        consumption_quintile[df.loc[product, 'EXIOBASE sector(s)']] += df.loc[product, quintile]/2
                        consumption_quintile[df.loc[product, df.columns[-3]]] += df.loc[product, quintile]/2
                except:
                    if np.isnan(df.loc[product, df.columns[-1]]):
                        consumption_quintile[df.loc[product, 'EXIOBASE sector(s)']] += df.loc[product, quintile]/3
                        consumption_quintile[df.loc[product, df.columns[-3]]] += df.loc[product, quintile]/3
                        consumption_quintile[df.loc[product, df.columns[-2]]] += df.loc[product, quintile]/3
    
    # add Canadian data for waste management triggered by households consumption
    waste_treatment_sectors = [i[1] for i in io.A.index if ('waste for treatment' in i[1] or 'landfill' in i[1] or 'sludge' in i[1]) and i[0] == 'CA']
    serie = (io.Y.loc[:,('CA','Final consumption expenditure by households')].groupby('sector').sum().loc[waste_treatment_sectors]/
            io.Y.loc[:,('CA','Final consumption expenditure by households')].sum().sum()*sum(consumption_quintile.values()))
    for index in serie.index:
        consumption_quintile[index] = serie.loc[index]
    
    # copy canadian consumption and scale the imports to the consumption of one household from Quebec
    dff = io.Y.loc[:, ('CA','Final consumption expenditure by households')].copy()
    total_commodity = {}
    foo = dff.groupby('sector').sum()
    for index in foo.index:
        total_commodity[index] = foo[index]
    for index in dff.index:
        if total_commodity[index[1]] != 0:
            dff[index] = dff[index] / total_commodity[index[1]] * consumption_quintile[index[1]]

    # just add 201 products for Quebec, to distinguish them from CA
    Y = pd.DataFrame(dff)
    Y.columns = ['Households Quebec']
    df_to_add = Y.loc['CA']
    df_to_add = df_to_add.append(pd.DataFrame(consumption_quintile['grid_mix'],index=[('QC','grid_mix')],columns=['Households Quebec']))
    df_to_add = df_to_add.set_index(pd.MultiIndex.from_product([['QC'],commodities]))
    Y = pd.concat([Y, df_to_add])
    Y.loc['CA'] = 0
    
    total_Y[quintile] = Y

np.save('final_demand',total_Y)

The resulting final demand vector is stored in a .npy format.